In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/mobile-price-classification/train.csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

In [ ]:
sns.pairplot(df)

In [ ]:
plt.figure(figsize=(17,6))
sns.heatmap(df.corr(), annot=True)

In [ ]:
def std_based(col_name,df):
    mean = df[col_name].mean()   #outliers
    std = df[col_name].std()
    cut_off = std * 3
    lower, upper = mean - cut_off, mean + cut_off
    new_df = df[(df[col_name] < upper) & (df[col_name] > lower)]
    return new_df

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, orient='h')

In [ ]:
df = std_based('px_height', df)

plt.figure(figsize=(12, 6))
sns.boxplot(data=df, orient='h')

In [ ]:
X = df.drop(['price_range','pc','three_g'], axis=1)
y = df['price_range']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Cassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
classification_report(y_test, y_pred)

In [ ]:
df.shape

# Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {'criterion': ('gini', 'entropy'),
         'splitter': ('best', 'random'),
         'max_depth': list(range(1, 20)),
         'min_samples_split':[2, 3, 4],
         'min_samples_leaf':(list(range(1, 20)))}


In [ ]:
tree_cv = GridSearchCV(model, param_grid = params, scoring='accuracy', n_jobs=-1, verbose=1, cv=3)

In [ ]:
tree_cv.fit(X_train, y_train)

In [ ]:
tree_cv.best_params_

In [ ]:
model = DecisionTreeClassifier(**tree_cv.best_params_)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)